In [1]:
from transformers import BertTokenizer
import datasets
import numpy as np
from transformers import AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification

import os
os.environ["WAND_DISABLED"] = "true"

dataset_multi_news = datasets.load_dataset("alexfabbri/multi_news", data_dir="content/datset",trust_remote_code=True)

c:\Users\Debdutta Chatterjee\Work\Project\Python\FineTuning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
def tokenize(data):
  input = tokenizer(data['document'],padding='max_length',truncation=True,max_length=512)
  output = tokenizer(data['summary'],padding ='max_length', truncation=True,max_length=128)
  input['labels'] = output['input_ids']
  #return input,output
  return input

dataset_multi_news_tokenize = dataset_multi_news.map(tokenize,batched=True)

train_dataset = dataset_multi_news_tokenize['train'].select(range(100))
train_dataset = train_dataset.map(lambda x: {"text": f"###user {x['document']} ###agent {x['summary']}"})

validation_dataset = dataset_multi_news_tokenize['validation'].select(range(10))
validation_dataset = validation_dataset.map(lambda x: {"text": f"###user {x['document']} ###agent {x['summary']}"})

from transformers import EncoderDecoderModel
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased','bert-base-uncased')
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.bos_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size
model.config.bos_token_id = tokenizer.cls_token_id

Map: 100%|██████████| 10/10 [00:00<00:00, 296.69 examples/s]
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight',

In [4]:
from transformers import TrainingArguments,Trainer

training_args = TrainingArguments(
    output_dir ='./results',
    #evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate = 5e-5,
    per_device_train_batch_size= 4,
    per_device_eval_batch_size= 4,
    num_train_epochs =3,
    weight_decay=0.01,
    report_to="none",
    use_cpu=False
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = validation_dataset
)
trainer.train()  

c:\Users\Debdutta Chatterjee\Work\Project\Python\FineTuning\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Debdutta Chatterjee\Work\Project\Python\FineTuning\.venv\Lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:631: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
c:\Users\Debdutta Chatterjee\Work\Project\Python\FineTuning\.venv\Lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the enco

Step,Training Loss


c:\Users\Debdutta Chatterjee\Work\Project\Python\FineTuning\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Debdutta Chatterjee\Work\Project\Python\FineTuning\.venv\Lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:631: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
c:\Users\Debdutta Chatterjee\Work\Project\Python\FineTuning\.venv\Lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the enco

TrainOutput(global_step=75, training_loss=6.367607421875, metrics={'train_runtime': 1125.0228, 'train_samples_per_second': 0.267, 'train_steps_per_second': 0.067, 'total_flos': 184036230144000.0, 'train_loss': 6.367607421875, 'epoch': 3.0})

In [7]:
from transformers import BertTokenizer

# Load the trained model
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
model.eval()  # Set the model to evaluation mode

def summarize(text):
    input_ids = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512).input_ids
    output = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True, decoder_start_token_id=model.config.decoder_start_token_id)
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary

# Example usage
document = f"###user {train_dataset[2]['document']}"
summary = summarize(document)
print("Generated Summary:", summary)


Generated Summary: – – – – the – – – the – –
